In [0]:
import sys, os
sys.path.append(os.pardir)

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import nltk
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import string

import textblob as tb
from tqdm import tqdm

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [0]:
# 데이터 읽기

colnames = ['comment', 'like', 'dislike', 'month']
# df = pd.read_csv('/content/gdrive/My Drive/Dongkuk_project/토막글 생성/LSTM/naver/naver_comments.csv')
df = pd.read_excel('LSTM/daum/daum_comments_1to8.xlsx')

df.columns = colnames
print(df.head())
print(len(df))

                                             comment  like  dislike  month
0         경제는 세계적인 흐름이라 시간이 걸릴거고\n문프님 잘하고 계십니다\n짱입니다  7487     1860      1
1  기레기만 헐뜯지않고 그대로 기사쓰도\n80프로는 됨\n공항ㆍ백화점가봐라\n경제는 완...  5245     1286      1
2  2018년 수출 6위국에 경제 망이라는 프레임은 정말 온세상이 비웃을 소리이다.\n...  1927      344      1
3  기발롬들이 경제공포심 조장해서 퍼트린효과지.수출이 사상최대를 찍고 내수가살아나고 조...  1378      256      1
4  언론이 왜 이렇게 503때와 다르게 냉소적인지 국민들은 알아야 한다. 기레기 적폐쓰...  1019      122      1
41035


In [0]:
# 공감비율 계산

def favor_cmt(l, d):
    return np.log(l+.5)/(np.log(l+d+1)+1)

df['empathize'] = df.apply(lambda x: favor_cmt(x['like'], x['dislike']), axis=1)

dat = df[(df['empathize'] >= 0.70) & (df['month']==7)] # 공감도 0.8 이상의 댓글만 가져옴
len(dat) # 가져온 후 댓글의 갯수

4135

In [0]:
comments = [] # 리스트 선언
comments.extend(list(dat.comment.values)) # 헤드라인의 값들을 리스트로 저장
comments = [n for n in comments if len(n) >=10]
len(comments) # 제거 후 샘플의 개수

4067

In [0]:
from string import punctuation
import re

def clean_text(text):
    cleaned_text = re.sub('[^가-힣]', ' ', text)
#     cleaned_text = re.sub('[0-9]', ' ', text)
#     cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"\'\·]',
#                           ' ', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    return cleaned_text

text = [clean_text(x) for x in comments]
text = [x.strip() for x in text]
text = pd.DataFrame(text, columns=['cmt'])
text[:10]

,cmt
0,조작 조작 조작 한마디로 자유사기꾼들의 일상은 아침에 일어나서 잠 들 때까지 조작 ...
1,자칭 보수라는 애국자들 대한민국 국민이 맞는지 의심이 간다
2,정말 가짜뉴스 어쩔수없는거냐
3,어쩌면 이렇게도 집요하게 한 나라의 대통령과 영부인의 흠을 잡기위해 발광을 하는가 ...
4,이걸 유튜브로 만이나 봤다니 진짜 국민들 정신 똑바로 차려야한다 저들은 아직 굳건하...
5,만 뷰 골빈 노인네들이 퍼나르기 카톡으로 주소 전송 전혀 이성적인 생각을 하지 못하...
6,남북미 정상회담이 얼마나 의미있는지에 대해서 이야기 하거나 앞으로 어떻게 될것인지 ...
7,문재인 김정숙 여사 까기에만 골몰하는 자위한국당과 쓰레기 찌라시들 너들과 같은 하늘...
8,유투브 가짜뉴스공장 심각하다 진짜 허위유포 엄벌해야한다
9,니들이 세운 바보 박구네가 해외가서 한 바보짓좀 다시 봐라


In [0]:
# 품사 사전 함수

from konlpy.tag import Okt
okt = Okt()
# okt.add_dictionary('손혜원', 'Noun')
# okt.add_dictionary('자한당', 'Noun')

def lemmatized(txt):
    a = []
    b = []
    a = okt.pos(txt)
    for word, tag in a :
        if tag in ['Noun', 'Adjective', 'Verb', 'Adverb']:
            b.append(word)
    return b

In [0]:
# 불용어 사전 로드

stopwords = pd.read_excel('stopwords100.xlsx', encoding='utf-8')
stop_words = list(stopwords['Column1'])
stop_words[99:]

['놓', '지금', '근데', '있는', '해서', '하는', '하지', '입니다', '한다', '하네', '하냐', '하니', '하나']

In [0]:
# 품사별 토큰화와 불용어 제거

tokenized_doc = text.apply(lambda row : lemmatized(row['cmt']), axis=1)
tokenized_doc = [x for x in tokenized_doc if x not in stop_words]
tokenized_doc[:5]

[['조작',
  '조작',
  '조작',
  '한마디',
  '자유',
  '사기꾼',
  '일상',
  '아침',
  '일어나서',
  '잠',
  '들',
  '때',
  '조작',
  '인생'],
 ['자칭', '보수', '애국자', '대한민국', '국민', '맞는지', '의심', '간다'],
 ['정말', '가짜', '뉴스', '수', '없는거냐'],
 ['어쩌면',
  '이렇게도',
  '집요하게',
  '한',
  '나라',
  '대통령',
  '영부인',
  '흠',
  '잡기',
  '위해',
  '발광',
  '하는가',
  '너희',
  '이',
  '나라',
  '국민',
  '아니란',
  '말',
  '아무리',
  '박근혜',
  '이명박',
  '지지',
  '다',
  '하더라도',
  '상식',
  '양심',
  '있을텐데',
  '너희',
  '사람',
  '아니란',
  '말',
  '무엇',
  '옳고',
  '그른',
  '분간',
  '하면서',
  '맹목',
  '소모전',
  '이제',
  '그만하길',
  '바랍니다'],
 ['걸',
  '유튜브',
  '만이',
  '봤다니',
  '진짜',
  '국민',
  '정신',
  '똑바로',
  '차려',
  '야한다',
  '저',
  '아직',
  '굳건',
  '하다',
  '총선',
  '반드시',
  '씨',
  '말리']]

In [0]:
# 역 토큰화

documents = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    documents.append(t)
    
documents[:5]

['조작 조작 조작 한마디 자유 사기꾼 일상 아침 일어나서 잠 들 때 조작 인생',
 '자칭 보수 애국자 대한민국 국민 맞는지 의심 간다',
 '정말 가짜 뉴스 수 없는거냐',
 '어쩌면 이렇게도 집요하게 한 나라 대통령 영부인 흠 잡기 위해 발광 하는가 너희 이 나라 국민 아니란 말 아무리 박근혜 이명박 지지 다 하더라도 상식 양심 있을텐데 너희 사람 아니란 말 무엇 옳고 그른 분간 하면서 맹목 소모전 이제 그만하길 바랍니다',
 '걸 유튜브 만이 봤다니 진짜 국민 정신 똑바로 차려 야한다 저 아직 굳건 하다 총선 반드시 씨 말리']

In [0]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx , topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words -1:-1]]))

In [0]:
no_features = 1000
tfidf_vectorizer = TfidfVectorizer(max_df = 0.95, min_df = 2,max_features=no_features)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [0]:
tf_vectorizer = CountVectorizer(max_df = 0.95, min_df = 2, max_features=no_features)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [0]:
no_topic = 5

In [0]:
nmf = NMF(n_components=no_topic, random_state = 1, alpha =.1, l1_ratio=.5, init = 'nndsvd').fit(tfidf)

In [0]:
lda = LatentDirichletAllocation(n_topics=no_topic, max_iter = 5, learning_method = 'online', learning_offset=50., random_state=0).fit(tf)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [0]:
no_top_words = 10
display_topics(nmf ,tfidf_feature_names, no_top_words)

Topic 0:
일본 한국 제품 불매운동 불매 여행 일본여행 합시다 금지 관광
Topic 1:
왜구 토착 한당 친일 자유 총선 박멸 본토 한국 하자
Topic 2:
국민 나라 하는 대통령 대한민국 사람 정부 문재인 한당 합니다
Topic 3:
아베 한국 트럼프 정부 나베 하는 같은 교활 일베 친일
Topic 4:
우리 주적 정부 불매운동 이번 보복 한다 하자 해야 대응


In [0]:
display_topics(lda , tf_feature_names , no_top_words)

Topic 0:
대통령 정권 국민 나라 문재인 하는 정치 사람 북한 쓰레기
Topic 1:
일본 한국 우리 아베 미국 나라 국가 하는 이번 경제
Topic 2:
국민 하는 지금 사람 하고 생각 정말 같은 소리 새끼
Topic 3:
한당 아베 친일 전쟁 정부 하는 인간 언론 자살 제발
Topic 4:
일본 왜구 우리 토착 국민 합니다 불매운동 하자 불매 제품
